In [2]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2

import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

from imutils import paths
import numpy as np
import os

print('Libraries Loaded')

Libraries Loaded


In [5]:
path = os.getcwd()+'/Dataset'
images = list(paths.list_images(path))
data = []
labels = []

# loop over the image paths
for image in images:
    # extract the class label from the filename
    label = image.split(os.path.sep)[-2]
    
    # load the input image (224x224) and preprocess it
    loaded_image = keras.preprocessing.image.load_img(image, target_size=(224, 224))
    loaded_image = keras.preprocessing.image.img_to_array(loaded_image)
    loaded_image = keras.applications.mobilenet_v2.preprocess_input(loaded_image)
    
    # update the data and labels lists, respectively
    data.append(loaded_image)
    labels.append(label)

data = np.array(data, dtype="float32")
labels = np.array(labels)

print(f'The shape of the Data is: {data.shape}')
print(f'The shape of the labels is: {labels.shape}')

The shape of the Data is: (2757, 224, 224, 3)
The shape of the labels is: (2757,)


In [6]:
encoder = LabelBinarizer()
labels_encoded = encoder.fit_transform(labels)

In [19]:
labels_cat = to_categorical(labels_encoded)

In [22]:
(x_train, x_test, y_train, y_test) = train_test_split(data, labels_cat,test_size=0.20, random_state=42)
print(f'The shape of X_train is {x_train.shape} and the shape of X_test is {x_test.shape}')
print(f'The shape of Y_train is {y_train.shape} and the shape of Y_test is {y_test.shape}')

The shape of X_train is (2205, 224, 224, 3) and the shape of X_test is (552, 224, 224, 3)
The shape of Y_train is (2205, 2) and the shape of Y_test is (552, 2)


In [17]:
aug = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,
                                                   height_shift_range=0.2,shear_range=0.15,
                                                   horizontal_flip=True,fill_mode="nearest")

In [19]:
# Using pre-trained model
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=keras.layers.Input(shape=(224, 224, 3)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/Users/saad/opt/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 3s 0us/step


In [22]:
# Remaining model
remainingModel = baseModel.output
remainingModel = keras.layers.AveragePooling2D(pool_size=(7, 7))(remainingModel)
remainingModel = keras.layers.Flatten(name="flatten")(remainingModel)
remainingModel = keras.layers.Dense(128, activation="relu")(remainingModel)
remainingModel = keras.layers.Dropout(0.5)(remainingModel)
remainingModel = keras.layers.Dense(2, activation="softmax")(remainingModel)

In [25]:
model = keras.Model(inputs=baseModel.input, outputs=remainingModel)

In [26]:
for layer in baseModel.layers:
    layer.trainable = False

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [28]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])

In [31]:
history = model.fit(aug.flow(x_train, y_train, batch_size=32),validation_data=(x_test, y_test),epochs=20)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
69/69 [==============================] - 84s 1s/step - loss: 0.4059 - acc: 0.8222 - val_loss: 0.1790 - val_acc: 0.9293
Epoch 2/20
69/69 [==============================] - 80s 1s/step - loss: 0.2444 - acc: 0.9098 - val_loss: 0.1416 - val_acc: 0.9493
Epoch 3/20
69/69 [==============================] - 86s 1s/step - loss: 0.1768 - acc: 0.9265 - val_loss: 0.1448 - val_acc: 0.9420
Epoch 4/20
69/69 [==============================] - 84s 1s/step - loss: 0.1927 - acc: 0.9261 - val_loss: 0.2038 - val_acc: 0.9130
Epoch 5/20
69/69 [==============================] - 79s 1s/step - loss: 0.1733 - acc: 0.9315 - val_loss: 0.1464 - val_acc: 0.9384
Epoch 6/20
69/69 [==============================] - 85s 1s/step - loss: 0.1636 - acc: 0.9351 - val_loss: 0.1124 - val_acc: 0.9547
Epoch 7/20
69/69 [==============================] - 79s 1s/step - loss: 0.1663 - acc: 0.9347 - val_loss: 0.1090 - val_acc: 0.9

69/69 [==============================] - 82s 1s/step - loss: 0.1241 - acc: 0.9533 - val_loss: 0.1395 - val_acc: 0.9475
Epoch 17/20
69/69 [==============================] - 82s 1s/step - loss: 0.1339 - acc: 0.9488 - val_loss: 0.1750 - val_acc: 0.9384
Epoch 18/20
69/69 [==============================] - 82s 1s/step - loss: 0.1424 - acc: 0.9447 - val_loss: 0.1267 - val_acc: 0.9438
Epoch 19/20
69/69 [==============================] - 82s 1s/step - loss: 0.1181 - acc: 0.9583 - val_loss: 0.1333 - val_acc: 0.9438
Epoch 20/20
69/69 [==============================] - 82s 1s/step - loss: 0.1172 - acc: 0.9624 - val_loss: 0.1744 - val_acc: 0.9348


In [23]:
model = keras.models.load_model('Mask_detector')
predIdxs = model.predict(x_test, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,target_names=encoder.classes_))


              precision    recall  f1-score   support

   with_mask       0.88      1.00      0.93       255
without_mask       1.00      0.88      0.94       297

    accuracy                           0.93       552
   macro avg       0.94      0.94      0.93       552
weighted avg       0.94      0.93      0.93       552



In [36]:
model.save('Mask_detector', save_format="h5")